In [1]:
!pip install multiSyncPy

import numpy as np
import scipy.spatial
import scipy.signal
import scipy.stats
import pandas as pd

from matplotlib import pyplot as plt

import multiSyncPy as msp
from multiSyncPy import synchrony_metrics as sm
print('done loading')

done loading


In [2]:
#Takes as input a list or array of multiple time series, outputs various metrics from the multiSyncPy package
def multiSyncPy_metrics(time_series):
    time_series = np.array(time_series)
    
    coherence_team = sm.coherence_team(time_series)
    
    symbolic_entropy = sm.symbolic_entropy(time_series)
    
    phases = np.angle(scipy.signal.hilbert(time_series))
    rho = sm.rho(phases)[1]
    
    recurrence_matrix = sm.recurrence_matrix(time_series, radius = 0.5)
    rqa_metrics = sm.rqa_metrics(recurrence_matrix)
    
    #Not sure if the np.tile parameters are correct
    time_series_sample = np.tile(time_series, (100, 1, 1)) + np.random.normal(0, 0.1, (100, len(time_series), len(time_series[0])))
    time_series_sample = np.angle(scipy.signal.hilbert(time_series_sample))
    
    #Do we want this one to be with the rest or do we want this to be separate, because the notebook mentions 
    #that this is for specific cases
    weak_null = sm.kuramoto_weak_null(time_series_sample)
    return coherence_team, symbolic_entropy, rho, weak_null

print('done')

done


In [3]:
#Hypothesis 1:
#extract relevant files Stroop fp1 and fp2

stroop_files_fp1=[]
stroop_files_fp2=[]

for i in range(1,41):
    f=pd.read_csv("Stroop/Stroop_sub_"+str(i)+".csv")
    stroop_files_fp1.append(f['2'])
    stroop_files_fp2.append(f['31'])
# print(stroop_files_fp1[0])
# print(stroop_files_fp2[0])
# print(len(stroop_files_fp1))
print('done')
print(stroop_files_fp1[0].shape)

done
(9600,)


In [ ]:
#calculate multiSyncPy metrics Stroop H1
coherenceList0=[]
entropyList0=[]
rhoList0=[]
weakNullList0=[]

for i in range(len(stroop_files_fp1)):
    print(i)
    inputArray = np.array((stroop_files_fp1[i], stroop_files_fp2[i]))
    coherence, entropy, rho, weakNull = multiSyncPy_metrics(inputArray)
    coherenceList0.append(coherence)
    entropyList0.append(entropy)
    rhoList0.append(rho)
    weakNullList0.append(weakNull)
    
print('done')

0
1
2
3
4


In [ ]:
#round lists

rounded_coherence0 = [round(num, 2) for num in coherenceList0]
rounded_entropy0 = [round(num, 2) for num in entropyList0]

df_h14 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': rounded_coherence0, 'Entropy': rounded_entropy0})
df_h14

In [ ]:
#convert df to csv to compare in H3

df_h1 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': coherenceList0, 'Entropy': entropyList0, 'Rho': rhoList0})

df_h1.to_csv('h1_stroop_fp1_fp2.csv',index=False)

In [ ]:
#mean values for Stroop fp1 and fp2 H1
print("Mean Coherence: ",np.mean(coherenceList0))
print("Mean Entropy: ",np.mean(entropyList0))
print("Mean Rho: ",np.mean(rhoList0))

In [ ]:
#extract relevant files Relax H1
relax_files_fp1=[]
relax_files_fp2=[]

for i in range(1,41):
    
    f=pd.read_csv("Relax/Relax_sub_"+str(i)+".csv")
    relax_files_fp1.append(f['2'])
    relax_files_fp2.append(f['31'])
# print(relax_files_fp1[0])
# print(relax_files_fp2[0])
# print(len(relax_files_fp1))

In [ ]:
#calculate multiSyncPy metrics Relax H1
coherenceList1=[]
entropyList1=[]
rhoList1=[]
weakNullList1=[]

for i in range(len(relax_files_fp1)):
    print(i)
    inputArray = np.array((relax_files_fp1[i], relax_files_fp2[i]))
    print(inputArray.shape)
    coherence, entropy, rho, weakNull = multiSyncPy_metrics(inputArray)
    coherenceList1.append(coherence)
    entropyList1.append(entropy)
    rhoList1.append(rho)
    weakNullList1.append(weakNull)

In [ ]:
#round lists

rounded_coherence1 = [round(num, 2) for num in coherenceList1]
rounded_entropy1 = [round(num, 2) for num in entropyList1]

df_h12 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': rounded_coherence1, 'Entropy': rounded_entropy1})
df_h12

In [ ]:
#convert df to csv to compare in H3

df_h13 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': coherenceList1, 'Entropy': entropyList1})
df_h13.to_csv('h1_relax_fp1_fp2.csv',index=False)

In [ ]:
#mean values for Relax fp1 and fp2 H1
print("Mean Coherence: ",np.mean(coherenceList1))
print("Mean Entropy: ",np.mean(entropyList1))
print("Mean Rho: ",np.mean(rhoList1))

In [ ]:
#significance test H1
from scipy.stats import ttest_ind

t_statistic, p_value = ttest_ind(coherenceList0, coherenceList1)

print('Coherence')
print("t-statistic:", t_statistic)
print("p-value:", p_value)

t_statistic, p_value = ttest_ind(entropyList0, entropyList1)

print()
print('Entropy')
print("t-statistic:", t_statistic)
print("p-value:", p_value)

# Surrogation technique 1: shuffling windows

In [ ]:
# For stroop data

surrogate_1_stroop_files_fp1 = []  
surrogate_1_stroop_files_fp2 = []

 
surrogate_1_stroop_fp1 = sm.shuffle_time_windows(np.array(stroop_files_fp1), 100)
surrogate_1_stroop_fp2 = sm.shuffle_time_windows(np.array(stroop_files_fp2), 100) 


for i in surrogate_1_stroop_fp1:
    surrogate_1_stroop_files_fp1.append(i)
    
for i in surrogate_1_stroop_fp2:
    surrogate_1_stroop_files_fp2.append(i)
    
print('done')

In [ ]:
#calculate multiSyncPy metrics Stroop H1
coherenceList0=[]
entropyList0=[]
rhoList0=[]
weakNullList0=[]

for i in range(len(surrogate_1_stroop_files_fp1)):
    print(i)
    inputArray = np.array((surrogate_1_stroop_files_fp1[i], surrogate_1_stroop_files_fp2[i]))
    coherence, entropy, rho, weakNull = multiSyncPy_metrics(inputArray)
    coherenceList0.append(coherence)
    entropyList0.append(entropy)
    rhoList0.append(rho)
    weakNullList0.append(weakNull)
    
print('done')

In [ ]:
#round lists

rounded_coherence0 = [round(num, 2) for num in coherenceList0]
rounded_entropy0 = [round(num, 2) for num in entropyList0]

df_h14 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': rounded_coherence0, 'Entropy': rounded_entropy0})
df_h14

In [ ]:
#convert df to csv to compare in H3

df_h1 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': coherenceList0, 'Entropy': entropyList0})

df_h1.to_csv('h1_surrogate1_stroop_fp1_fp2.csv',index=False)

In [ ]:
#mean values for Stroop fp1 and fp2 H1
print("Mean Coherence: ",np.mean(coherenceList0))
print("Mean Entropy: ",np.mean(entropyList0))
print("Mean Rho: ",np.mean(rhoList0))

In [ ]:
# For relax data

surrogate_1_relax_files_fp1 = []  
surrogate_1_relax_files_fp2 = []

 
surrogate_1_relax_fp1 = sm.shuffle_time_windows(np.array(relax_files_fp1), 100)
surrogate_1_relax_fp2 = sm.shuffle_time_windows(np.array(relax_files_fp2), 100)   


for i in surrogate_1_relax_fp1:
    surrogate_1_relax_files_fp1.append(i)
    
for i in surrogate_1_stroop_fp2:
    surrogate_1_relax_files_fp2.append(i)
    
print('done')

In [ ]:
#calculate multiSyncPy metrics Relax H1
coherenceList0=[]
entropyList0=[]
rhoList0=[]
weakNullList0=[]

for i in range(len(surrogate_1_relax_files_fp1)):
    print(i)
    inputArray = np.array((surrogate_1_relax_files_fp1[i], surrogate_1_relax_files_fp2[i]))
    coherence, entropy, rho, weakNull = multiSyncPy_metrics(inputArray)
    coherenceList0.append(coherence)
    entropyList0.append(entropy)
    rhoList0.append(rho)
    weakNullList0.append(weakNull)
    
print('done')

In [ ]:
#round lists

rounded_coherence0 = [round(num, 2) for num in coherenceList0]
rounded_entropy0 = [round(num, 2) for num in entropyList0]

df_h14 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': rounded_coherence0, 'Entropy': rounded_entropy0})
df_h14


In [ ]:
#convert df to csv to compare in H3

df_h1 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': coherenceList0, 'Entropy': entropyList0})

df_h1.to_csv('h1_surrogate1_relax_fp1_fp2.csv',index=False)

In [ ]:
#mean values for Stroop fp1 and fp2 H1
print("Mean Coherence: ",np.mean(coherenceList0))
print("Mean Entropy: ",np.mean(entropyList0))
print("Mean Rho: ",np.mean(rhoList0))

# Surrogation technique 2: Data sliding

In [ ]:
# for stroop data

# Define the percentage of data to slide
slide_percentage = 0.6

# Define a function for data sliding
def slide_data(data, slide_percentage):
    # Determine the number of data points to slide
    num_points_to_slide = int(len(data) * slide_percentage)
    
    # Slide the data
    slid_data = np.roll(data, num_points_to_slide)
    
    return slid_data

surrogate_2_stroop_files_fp1 = []
surrogate_2_stroop_files_fp2 = []

for data_fp1 in stroop_files_fp1:
    slid_data = slide_data(data_fp1.values, slide_percentage)
    surrogate_2_stroop_files_fp1.append(slid_data)
    
for data_fp2 in stroop_files_fp2:
    slid_data = slide_data(data_fp2.values, slide_percentage)
    surrogate_2_stroop_files_fp2.append(slid_data)
    
print('done')

In [ ]:
#calculate multiSyncPy metrics Stroop H1
coherenceList0=[]
entropyList0=[]
rhoList0=[]
weakNullList0=[]

for i in range(len(surrogate_2_stroop_files_fp1)):
    print(i)
    inputArray = np.array((surrogate_2_stroop_files_fp1[i], surrogate_2_stroop_files_fp2[i]))
    coherence, entropy, rho, weakNull = multiSyncPy_metrics(inputArray)
    coherenceList0.append(coherence)
    entropyList0.append(entropy)
    rhoList0.append(rho)
    weakNullList0.append(weakNull)
    
print('done')

In [ ]:
#round lists

rounded_coherence0 = [round(num, 2) for num in coherenceList0]
rounded_entropy0 = [round(num, 2) for num in entropyList0]

df_h14 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': rounded_coherence0, 'Entropy': rounded_entropy0})
df_h14

In [ ]:
#convert df to csv to compare in H3

df_h1 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': coherenceList0, 'Entropy': entropyList0})

df_h1.to_csv('h1_surrogate2_stroop_fp1_fp2.csv',index=False)

In [ ]:
#mean values for Stroop fp1 and fp2 H1
print("Mean Coherence: ",np.mean(coherenceList0))
print("Mean Entropy: ",np.mean(entropyList0))
print("Mean Rho: ",np.mean(rhoList0))

In [ ]:
# for relax data

# Define the percentage of data to slide
slide_percentage = 0.6

# Define a function for data sliding
def slide_data(data, slide_percentage):
    # Determine the number of data points to slide
    num_points_to_slide = int(len(data) * slide_percentage)
    
    # Slide the data
    slid_data = np.roll(data, num_points_to_slide)
    
    return slid_data

surrogate_2_relax_files_fp1 = []
surrogate_2_relax_files_fp2 = []

for data_fp1 in relax_files_fp1:
    slid_data = slide_data(data_fp1.values, slide_percentage)
    surrogate_2_relax_files_fp1.append(slid_data)
    
for data_fp2 in relax_files_fp2:
    slid_data = slide_data(data_fp2.values, slide_percentage)
    surrogate_2_relax_files_fp2.append(slid_data)
    
print('done')

In [ ]:
#calculate multiSyncPy metrics Relax H1
coherenceList0=[]
entropyList0=[]
rhoList0=[]
weakNullList0=[]

for i in range(len(surrogate_2_relax_files_fp1)):
    print(i)
    inputArray = np.array((surrogate_2_relax_files_fp1[i], surrogate_2_relax_files_fp2[i]))
    coherence, entropy, rho, weakNull = multiSyncPy_metrics(inputArray)
    coherenceList0.append(coherence)
    entropyList0.append(entropy)
    rhoList0.append(rho)
    weakNullList0.append(weakNull)
    
print('done')

In [ ]:
#round lists

rounded_coherence0 = [round(num, 2) for num in coherenceList0]
rounded_entropy0 = [round(num, 2) for num in entropyList0]

df_h14 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': rounded_coherence0, 'Entropy': rounded_entropy0})
df_h14

In [ ]:
#convert df to csv to compare in H3

df_h1 = pd.DataFrame({'Participant': list(range(1,41)), 'Coherence': coherenceList0, 'Entropy': entropyList0})

df_h1.to_csv('h1_surrogate2_relax_fp1_fp2.csv',index=False)

In [ ]:
#mean values for Relax fp1 and fp2 H1
print("Mean Coherence: ",np.mean(coherenceList0))
print("Mean Entropy: ",np.mean(entropyList0))
print("Mean Rho: ",np.mean(rhoList0))